In [11]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import random

In [12]:
# Fix random seed
seed = 42
np.random.seed(seed)
random.seed(seed)

# Pre-processing input data

In [13]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [14]:
results_df = pd.read_csv('../results/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [15]:
# remove one computer for testing
g_train = results_df[(results_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['benchmark','brand_raw'])
g_test = results_df[results_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw'])

In [16]:
mm_df = results_df[results_df['benchmark']=='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw'])
mm_test = mm_df[mm_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [17]:
st_df = results_df[results_df['benchmark']!='MATRIX_MULT'].drop(columns=['benchmark','count'])
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw'])
st_test = st_df[st_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [18]:
target = 'total_time'
features = mm_test.columns.copy().drop(target)
features_st = features.copy().drop(['count'])

In [19]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler = StandardScaler()
X_g_train = scaler.fit_transform(X_g_train)
X_g_test = scaler.transform(X_g_test)

In [20]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_st_train = scaler.fit_transform(X_st_train)
X_st_test = scaler.transform(X_st_test)

In [21]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler = StandardScaler()
X_mm_train = scaler.fit_transform(X_mm_train)
X_mm_test = scaler.transform(X_mm_test)

# Model

In [22]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test):
	dtrain = xgb.DMatrix(X_train, label=y_train)
	dtest = xgb.DMatrix(X_test, label=y_test)
	
	# Definimos los hiperparámetros a buscar (XGBoost)
	param = {
		"verbosity": 0,
		"device": "gpu",
		"objective": "reg:squarederror",
		"eval_metric": "rmse",
		"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
		# L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
		"learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
	}
    
	if param["booster"] in ["gbtree", "dart"]:
		# maximum depth of the tree, signifies complexity of the tree.
		param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
		# minimum child weight, larger the term more conservative the tree.
		param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
		param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
		# defines how selective algorithm is.
		param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
		param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

	if param["booster"] == "dart":
		param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
		param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
		param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
		param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

	# training
	model = xgb.train(param, dtrain)
	# evaluation
	preds = model.predict(dtest)
	mse = mean_squared_error(y_test, preds)
	rmse = np.sqrt(mse)
	print(f"Trial: {trial.number} - MSE: {mse} - RMSE: {rmse}")
	return mse

# Hyperparameters Optimization

In [23]:
n_trials = 100

## General

In [24]:
# configuration optuna
study_g = optuna.create_study(direction='minimize')
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test), n_trials=n_trials)

[I 2024-06-20 05:07:53,392] A new study created in memory with name: no-name-1c8d10c1-3bef-47bb-89ef-e9a6b726d669
[I 2024-06-20 05:07:53,592] Trial 0 finished with value: 171.2254572160379 and parameters: {'booster': 'dart', 'lambda': 0.0014538556273497544, 'alpha': 5.005889390488909e-08, 'learning_rate': 0.00865619578568476, 'subsample': 0.9587051478713837, 'colsample_bytree': 0.7889244777924388, 'max_depth': 7, 'min_child_weight': 6, 'eta': 1.6138234251349032e-06, 'gamma': 0.010815874385654697, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0173657362888369, 'skip_drop': 2.732516204567645e-06}. Best is trial 0 with value: 171.2254572160379.
[I 2024-06-20 05:07:53,762] Trial 1 finished with value: 168.71171360893612 and parameters: {'booster': 'gbtree', 'lambda': 0.026872276086811094, 'alpha': 2.7942780675855837e-05, 'learning_rate': 0.00628063562725784, 'subsample': 0.7972219093379038, 'colsample_bytree': 0.7962362024012553, 'max_depth'

Trial: 0 - MSE: 171.2254572160379 - RMSE: 13.085314563129078
Trial: 1 - MSE: 168.71171360893612 - RMSE: 12.988907329292026


[I 2024-06-20 05:07:54,270] Trial 2 finished with value: 178.08865306568958 and parameters: {'booster': 'gblinear', 'lambda': 0.28093897148837876, 'alpha': 0.18726695981703867, 'learning_rate': 0.014491129416657571, 'subsample': 0.7415399365221185, 'colsample_bytree': 0.10893446818504315}. Best is trial 1 with value: 168.71171360893612.
[I 2024-06-20 05:07:54,364] Trial 3 finished with value: 167.64277267047572 and parameters: {'booster': 'dart', 'lambda': 1.0207862562662872e-05, 'alpha': 6.461239014938857e-05, 'learning_rate': 0.005811398665888791, 'subsample': 0.5878046603201001, 'colsample_bytree': 0.5451890208576909, 'max_depth': 5, 'min_child_weight': 6, 'eta': 0.5803899445435506, 'gamma': 2.94172854755518e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 3.9508162689171725e-07, 'skip_drop': 0.019202284433642733}. Best is trial 3 with value: 167.64277267047572.
[I 2024-06-20 05:07:54,399] Trial 4 finished with value: 193.83336762732

Trial: 2 - MSE: 178.08865306568958 - RMSE: 13.34498606464951
Trial: 3 - MSE: 167.64277267047572 - RMSE: 12.947693720137023
Trial: 4 - MSE: 193.8333676273222 - RMSE: 13.922405238582957
Trial: 5 - MSE: 162.90417522425628 - RMSE: 12.763391995243909


[I 2024-06-20 05:07:54,489] Trial 6 finished with value: 160.92714776512258 and parameters: {'booster': 'gbtree', 'lambda': 0.009256376134490079, 'alpha': 1.985052717342336e-07, 'learning_rate': 0.0033861712469100552, 'subsample': 0.39512026456355626, 'colsample_bytree': 0.11362975389046022, 'max_depth': 7, 'min_child_weight': 2, 'eta': 6.574699967564232e-06, 'gamma': 1.2278869421701564e-05, 'grow_policy': 'depthwise'}. Best is trial 6 with value: 160.92714776512258.
[I 2024-06-20 05:07:54,517] Trial 7 finished with value: 161.72305092432111 and parameters: {'booster': 'dart', 'lambda': 1.3618834439693241e-05, 'alpha': 0.00028474607604740714, 'learning_rate': 0.0013725768158385067, 'subsample': 0.9150937850508525, 'colsample_bytree': 0.4452448750505416, 'max_depth': 5, 'min_child_weight': 8, 'eta': 0.01325239508464732, 'gamma': 8.333125376426894e-07, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0005293693416500986, 'skip_drop': 6.243474

...

Trial: 91 - MSE: 160.27305531572645 - RMSE: 12.659899498642414
Trial: 92 - MSE: 160.3555481052056 - RMSE: 12.663157114448419
Trial: 93 - MSE: 160.34291460820617 - RMSE: 12.662658275741558
Trial: 94 - MSE: 160.32811462954115 - RMSE: 12.66207386763879


[I 2024-06-20 05:08:00,871] Trial 95 finished with value: 160.50515280664155 and parameters: {'booster': 'gbtree', 'lambda': 4.746510289951108e-05, 'alpha': 0.6081459199016822, 'learning_rate': 0.0017003176133927036, 'subsample': 0.3038964549420372, 'colsample_bytree': 0.11544943527613949, 'max_depth': 3, 'min_child_weight': 4, 'eta': 0.019462721629226142, 'gamma': 0.04176124805877608, 'grow_policy': 'depthwise'}. Best is trial 27 with value: 160.26098623673863.
[I 2024-06-20 05:08:00,937] Trial 96 finished with value: 161.195652772599 and parameters: {'booster': 'dart', 'lambda': 0.00019032559154164437, 'alpha': 0.15577180105162325, 'learning_rate': 0.0012097399515155962, 'subsample': 0.256961798728503, 'colsample_bytree': 0.26502651564813295, 'max_depth': 3, 'min_child_weight': 3, 'eta': 0.00113499973082932, 'gamma': 0.062049342472083074, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 2.2945698960797716e-05, 'skip_drop': 5.32344236523480

Trial: 95 - MSE: 160.50515280664155 - RMSE: 12.66906282274429
Trial: 96 - MSE: 161.195652772599 - RMSE: 12.696284998872663
Trial: 97 - MSE: 160.36106081025918 - RMSE: 12.663374779665142
Trial: 98 - MSE: 160.52014813732464 - RMSE: 12.669654617917752


[I 2024-06-20 05:08:01,080] Trial 99 finished with value: 217.8455056356901 and parameters: {'booster': 'gbtree', 'lambda': 1.7751110451511513e-05, 'alpha': 0.20143133994757134, 'learning_rate': 0.06994120043615074, 'subsample': 0.3272267924456955, 'colsample_bytree': 0.24023758726999808, 'max_depth': 3, 'min_child_weight': 3, 'eta': 0.00037528727976388017, 'gamma': 0.9555116770771628, 'grow_policy': 'lossguide'}. Best is trial 27 with value: 160.26098623673863.


Trial: 99 - MSE: 217.8455056356901 - RMSE: 14.759590293625704


In [25]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 27
Mejores parametros: {'booster': 'gbtree', 'lambda': 0.0036413406830877915, 'alpha': 0.00014215649240147834, 'learning_rate': 0.0010022326752607113, 'subsample': 0.39041476377585127, 'colsample_bytree': 0.12960686099491403, 'max_depth': 3, 'min_child_weight': 2, 'eta': 0.00017675419801249995, 'gamma': 0.0009494931173712514, 'grow_policy': 'lossguide'}
Mejor valor de pérdida: 160.26098623673863


## Single Thread

In [26]:
# configuration optuna
study_st = optuna.create_study(direction='minimize')
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test), n_trials=n_trials)

[I 2024-06-20 05:08:01,120] A new study created in memory with name: no-name-fcfa30cc-ac63-41b4-be13-1918a6202e83
[I 2024-06-20 05:08:01,183] Trial 0 finished with value: 45.2575714858915 and parameters: {'booster': 'gbtree', 'lambda': 0.1867653655375503, 'alpha': 6.725120080279762e-05, 'learning_rate': 0.00537942525432398, 'subsample': 0.27188392153356294, 'colsample_bytree': 0.5584075605314006, 'max_depth': 3, 'min_child_weight': 2, 'eta': 0.00018881765575451637, 'gamma': 0.00706573735615031, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 45.2575714858915.
[I 2024-06-20 05:08:01,217] Trial 1 finished with value: 76.78183095708712 and parameters: {'booster': 'gbtree', 'lambda': 0.000939626610510311, 'alpha': 0.001268096498698477, 'learning_rate': 0.0872018341420834, 'subsample': 0.8623682644148476, 'colsample_bytree': 0.53228595411604, 'max_depth': 7, 'min_child_weight': 7, 'eta': 0.006365213269504912, 'gamma': 0.0022364440167081255, 'grow_policy': 'lossguide'}. Best is tria

Trial: 0 - MSE: 45.2575714858915 - RMSE: 6.727374784110924
Trial: 1 - MSE: 76.78183095708712 - RMSE: 8.762524234322386
Trial: 2 - MSE: 45.32966193546228 - RMSE: 6.73273064480247


[I 2024-06-20 05:08:01,297] Trial 3 finished with value: 44.94903614326213 and parameters: {'booster': 'dart', 'lambda': 0.006069103315977123, 'alpha': 0.004584945489306408, 'learning_rate': 0.011394444516075276, 'subsample': 0.29205100960285907, 'colsample_bytree': 0.1545350484920378, 'max_depth': 9, 'min_child_weight': 9, 'eta': 2.8389592793005736e-07, 'gamma': 0.0007896135237342103, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.0031377171648738636, 'skip_drop': 1.907103264551179e-05}. Best is trial 3 with value: 44.94903614326213.
[I 2024-06-20 05:08:01,312] Trial 4 finished with value: 44.60443536342406 and parameters: {'booster': 'gbtree', 'lambda': 0.0011736490140868807, 'alpha': 2.4190243119375894e-07, 'learning_rate': 0.0025533334296562616, 'subsample': 0.7024132381655763, 'colsample_bytree': 0.5098436269349041, 'max_depth': 3, 'min_child_weight': 7, 'eta': 0.008048854459140515, 'gamma': 0.06328233220063201, 'grow_policy': 'dept

Trial: 3 - MSE: 44.94903614326213 - RMSE: 6.704404234774491
Trial: 4 - MSE: 44.60443536342406 - RMSE: 6.678655206209111


[I 2024-06-20 05:08:01,329] Trial 5 finished with value: 44.151258283532094 and parameters: {'booster': 'gbtree', 'lambda': 0.0018074740451532836, 'alpha': 0.3156639162233236, 'learning_rate': 0.001682376416692792, 'subsample': 0.32530112853181037, 'colsample_bytree': 0.7110017589293898, 'max_depth': 5, 'min_child_weight': 8, 'eta': 1.2164166540000353e-06, 'gamma': 0.04169452333057351, 'grow_policy': 'lossguide'}. Best is trial 5 with value: 44.151258283532094.
[I 2024-06-20 05:08:01,342] Trial 6 finished with value: 44.63433406658219 and parameters: {'booster': 'gbtree', 'lambda': 0.00019133494875660551, 'alpha': 1.645904824951664e-05, 'learning_rate': 0.005973624953559819, 'subsample': 0.24522646422555888, 'colsample_bytree': 0.4211442873224067, 'max_depth': 3, 'min_child_weight': 10, 'eta': 0.11243495238071785, 'gamma': 0.0036616389871517887, 'grow_policy': 'lossguide'}. Best is trial 5 with value: 44.151258283532094.
[I 2024-06-20 05:08:01,378] Trial 7 finished with value: 48.13717

...

Trial: 92 - MSE: 43.93757266779656 - RMSE: 6.62854227321487
Trial: 93 - MSE: 43.75738721705454 - RMSE: 6.614936675211226
Trial: 94 - MSE: 43.77948162278291 - RMSE: 6.616606503547185


[I 2024-06-20 05:08:07,047] Trial 95 finished with value: 58.48920552027598 and parameters: {'booster': 'dart', 'lambda': 0.0015547916136135678, 'alpha': 5.0576019791781e-08, 'learning_rate': 0.09913175188404053, 'subsample': 0.40846882678904506, 'colsample_bytree': 0.27527432563722837, 'max_depth': 3, 'min_child_weight': 9, 'eta': 4.517349732032842e-08, 'gamma': 0.0013503358134479078, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0011506504777227078, 'skip_drop': 0.060048241033038345}. Best is trial 93 with value: 43.75738721705454.
[I 2024-06-20 05:08:07,126] Trial 96 finished with value: 43.88997506609864 and parameters: {'booster': 'dart', 'lambda': 0.0004592983692121953, 'alpha': 1.1610552259364719e-06, 'learning_rate': 0.00207927216428196, 'subsample': 0.3673956013809959, 'colsample_bytree': 0.15393429650221133, 'max_depth': 3, 'min_child_weight': 9, 'eta': 1.3486240348550466e-07, 'gamma': 0.0032532294175241345, 'grow_policy': 'los

Trial: 95 - MSE: 58.48920552027598 - RMSE: 7.647823580619259
Trial: 96 - MSE: 43.88997506609864 - RMSE: 6.624950948203212
Trial: 97 - MSE: 43.79746548972348 - RMSE: 6.617965358758195


[I 2024-06-20 05:08:07,293] Trial 98 finished with value: 43.86429643482402 and parameters: {'booster': 'dart', 'lambda': 0.1249696496803101, 'alpha': 1.6691548996298033e-07, 'learning_rate': 0.0015217936554044935, 'subsample': 0.4491429230627066, 'colsample_bytree': 0.34113050274727, 'max_depth': 5, 'min_child_weight': 10, 'eta': 6.066223954803758e-08, 'gamma': 0.0008529234333212736, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.08755222633058918, 'skip_drop': 0.16997851091206043}. Best is trial 93 with value: 43.75738721705454.
[I 2024-06-20 05:08:07,362] Trial 99 finished with value: 46.56677110874584 and parameters: {'booster': 'dart', 'lambda': 0.4337294737418141, 'alpha': 4.7411794989053646e-08, 'learning_rate': 0.013320723129777168, 'subsample': 0.4322467339945157, 'colsample_bytree': 0.39618832277004334, 'max_depth': 5, 'min_child_weight': 10, 'eta': 3.39168911048647e-08, 'gamma': 0.000453214640857378, 'grow_policy': 'lossguide',

Trial: 98 - MSE: 43.86429643482402 - RMSE: 6.623012640394402
Trial: 99 - MSE: 46.56677110874584 - RMSE: 6.823984987435556


In [27]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 93
Mejores parametros: {'booster': 'dart', 'lambda': 0.0017586206157034748, 'alpha': 4.50273184369549e-08, 'learning_rate': 0.0013121789063029298, 'subsample': 0.41213734157363835, 'colsample_bytree': 0.1611597037187561, 'max_depth': 3, 'min_child_weight': 10, 'eta': 4.641153223811685e-08, 'gamma': 0.0006872086181180318, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.3083535111007091, 'skip_drop': 0.08125508254588887}
Mejor valor de pérdida en validación: 43.75738721705454


## Multi Thread

In [28]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize')
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test), n_trials=n_trials)

[I 2024-06-20 05:08:07,395] A new study created in memory with name: no-name-222650d0-f7c5-40d7-8a3d-3b9fd630d5d3
[I 2024-06-20 05:08:07,563] Trial 0 finished with value: 645.0482203049585 and parameters: {'booster': 'gblinear', 'lambda': 1.566642925568928e-05, 'alpha': 9.897925502311528e-06, 'learning_rate': 0.001734402993756365, 'subsample': 0.3536040171548212, 'colsample_bytree': 0.7432931079469848}. Best is trial 0 with value: 645.0482203049585.


Trial: 0 - MSE: 645.0482203049585 - RMSE: 25.3977995169849


[I 2024-06-20 05:08:07,595] Trial 1 finished with value: 610.7901270340824 and parameters: {'booster': 'gblinear', 'lambda': 2.8255132933587186e-07, 'alpha': 0.5788843170351062, 'learning_rate': 0.016421281349645316, 'subsample': 0.2925479624388597, 'colsample_bytree': 0.5906581923808469}. Best is trial 1 with value: 610.7901270340824.
[I 2024-06-20 05:08:07,603] Trial 2 finished with value: 646.9861450659603 and parameters: {'booster': 'gblinear', 'lambda': 0.00019661514245719773, 'alpha': 2.4092625266629867e-05, 'learning_rate': 0.0011290062510884605, 'subsample': 0.23588512886395752, 'colsample_bytree': 0.22812292461155542}. Best is trial 1 with value: 610.7901270340824.
[I 2024-06-20 05:08:07,643] Trial 3 finished with value: 626.8205511938669 and parameters: {'booster': 'gblinear', 'lambda': 7.609571060522389e-08, 'alpha': 4.4330488210452565e-05, 'learning_rate': 0.00914103962755503, 'subsample': 0.9990809819066233, 'colsample_bytree': 0.9786686366570218}. Best is trial 1 with val

Trial: 1 - MSE: 610.7901270340824 - RMSE: 24.714168548306098
Trial: 2 - MSE: 646.9861450659603 - RMSE: 25.43592233566458
Trial: 3 - MSE: 626.8205511938669 - RMSE: 25.03638454717188
Trial: 4 - MSE: 655.4280420053002 - RMSE: 25.601328910923748
Trial: 5 - MSE: 658.7888563802878 - RMSE: 25.666882482691346
Trial: 6 - MSE: 664.6696777650578 - RMSE: 25.78118844749128
Trial: 7 - MSE: 656.5151700811832 - RMSE: 25.622551982212528


[I 2024-06-20 05:08:07,777] Trial 8 finished with value: 668.1667104701007 and parameters: {'booster': 'dart', 'lambda': 7.783117071354533e-07, 'alpha': 0.00015061563985039662, 'learning_rate': 0.013407487242460197, 'subsample': 0.25355002391081594, 'colsample_bytree': 0.5274180885262784, 'max_depth': 3, 'min_child_weight': 6, 'eta': 0.0627102818824803, 'gamma': 1.6003479591156951e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 3.226580916099858e-05, 'skip_drop': 0.5039660803710911}. Best is trial 1 with value: 610.7901270340824.


...

Trial: 89 - MSE: 866.3935977862511 - RMSE: 29.43456467804902
Trial: 90 - MSE: 837.3902156151595 - RMSE: 28.93769540953736
Trial: 91 - MSE: 597.04254244203 - RMSE: 24.43445400335416
Trial: 92 - MSE: 621.4440815987233 - RMSE: 24.928780186738447


[I 2024-06-20 05:08:11,977] Trial 93 finished with value: 607.7031460596679 and parameters: {'booster': 'gblinear', 'lambda': 1.9858007644924797e-06, 'alpha': 0.3285700938809937, 'learning_rate': 0.05178149318853767, 'subsample': 0.8584297273396827, 'colsample_bytree': 0.8899008147020082}. Best is trial 77 with value: 583.1702479804816.
[I 2024-06-20 05:08:12,017] Trial 94 finished with value: 609.2641026992313 and parameters: {'booster': 'gblinear', 'lambda': 7.07323116610708e-06, 'alpha': 0.5897371206476705, 'learning_rate': 0.08711443607547491, 'subsample': 0.949592388040684, 'colsample_bytree': 0.8600149029294692}. Best is trial 77 with value: 583.1702479804816.
[I 2024-06-20 05:08:12,056] Trial 95 finished with value: 587.1036661195039 and parameters: {'booster': 'gblinear', 'lambda': 7.853130136287241e-07, 'alpha': 0.9390806990962934, 'learning_rate': 0.03991450854830915, 'subsample': 0.5845736784089661, 'colsample_bytree': 0.7763599152542677}. Best is trial 77 with value: 583.17

Trial: 93 - MSE: 607.7031460596679 - RMSE: 24.651635768436705
Trial: 94 - MSE: 609.2641026992313 - RMSE: 24.683275769217328
Trial: 95 - MSE: 587.1036661195039 - RMSE: 24.230222164055863
Trial: 96 - MSE: 619.7585804100003 - RMSE: 24.89495090193994
Trial: 97 - MSE: 584.7984883914664 - RMSE: 24.182607146283182
Trial: 98 - MSE: 599.9249484640748 - RMSE: 24.49336539685951


[I 2024-06-20 05:08:12,203] Trial 99 finished with value: 615.0541085942702 and parameters: {'booster': 'gblinear', 'lambda': 2.881260857862524e-07, 'alpha': 0.06276387768838092, 'learning_rate': 0.047655949060169206, 'subsample': 0.587396885455948, 'colsample_bytree': 0.783600333855249}. Best is trial 77 with value: 583.1702479804816.


Trial: 99 - MSE: 615.0541085942702 - RMSE: 24.80028444583389


In [29]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 77
Mejores parametros: {'booster': 'gblinear', 'lambda': 3.7493443285470513e-06, 'alpha': 0.9996777184324622, 'learning_rate': 0.0610679855567703, 'subsample': 0.7971001304451883, 'colsample_bytree': 0.8372095252299435}
Mejor valor de pérdida en validación: 583.1702479804816


# Training

## General

In [30]:
# hyperparameters
study_g.best_trial.params

{'booster': 'gbtree',
 'lambda': 0.0036413406830877915,
 'alpha': 0.00014215649240147834,
 'learning_rate': 0.0010022326752607113,
 'subsample': 0.39041476377585127,
 'colsample_bytree': 0.12960686099491403,
 'max_depth': 3,
 'min_child_weight': 2,
 'eta': 0.00017675419801249995,
 'gamma': 0.0009494931173712514,
 'grow_policy': 'lossguide'}

In [31]:
# general models
dtrain = xgb.DMatrix(X_g_train, label=y_g_train)
dtest = xgb.DMatrix(X_g_test, label=y_g_test)
# training
model = xgb.train(study_g.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_g_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 160.36008315224154 - RMSE: 12.663336177810393


## Single Thread

In [32]:
# hyperparameters
study_st.best_trial.params

{'booster': 'dart',
 'lambda': 0.0017586206157034748,
 'alpha': 4.50273184369549e-08,
 'learning_rate': 0.0013121789063029298,
 'subsample': 0.41213734157363835,
 'colsample_bytree': 0.1611597037187561,
 'max_depth': 3,
 'min_child_weight': 10,
 'eta': 4.641153223811685e-08,
 'gamma': 0.0006872086181180318,
 'grow_policy': 'lossguide',
 'sample_type': 'uniform',
 'normalize_type': 'tree',
 'rate_drop': 0.3083535111007091,
 'skip_drop': 0.08125508254588887}

In [33]:
# single thread model
dtrain = xgb.DMatrix(X_st_train, label=y_st_train)
dtest = xgb.DMatrix(X_st_test, label=y_st_test)
# training
model = xgb.train(study_st.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_st_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 43.79687484967787 - RMSE: 6.617920734617321


## Multi Thread

In [34]:
# hyperparameters
study_mm.best_trial.params

{'booster': 'gblinear',
 'lambda': 3.7493443285470513e-06,
 'alpha': 0.9996777184324622,
 'learning_rate': 0.0610679855567703,
 'subsample': 0.7971001304451883,
 'colsample_bytree': 0.8372095252299435}

In [35]:
# multi thread model
dtrain = xgb.DMatrix(X_mm_train, label=y_mm_train)
dtest = xgb.DMatrix(X_mm_test, label=y_mm_test)
# training
model = xgb.train(study_mm.best_trial.params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(y_mm_test, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)}")

MSE: 582.919569284892 - RMSE: 24.14372732791878


# Conclusion
Queda trabajo que hacer en la red, además de conseguir más datos para un entrenamiento más robusto. Queda por ahora descartado el uso de solo un modelo para multi-threading y single-threading, ya que el modelo tiene más del triple de *loss*.